##Updated HLCM estimation

Sam Maurer and Paul Waddell, August 2015

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os; os.chdir('..');os.chdir('..')
import models
import orca
# to try variations, create a new yaml file and run this to register it
# to save variations, create a new yaml file and run this to register it
from urbansim.utils import misc
from urbansim import accounts
from urbansim.developer import sqftproforma
from urbansim_defaults import models
from urbansim_defaults import utils

In [15]:
%%capture
orca.run(["neighborhood_vars"])

In [16]:
%%capture
orca.run(["rsh_simulate", "rrh_simulate", "nrh_simulate"])

## ols

In [4]:
import numpy as np
import pandas
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [17]:
# a is buildings
bb = orca.get_table('buildings').to_frame()[['node_id','residential_price','sqft_per_unit','zone_id']]
bb.head()

,node_id,residential_price,sqft_per_unit,zone_id
building_id,,,,
1,11584,392.395039,794,893
2,11584,284.880588,1600,893
3,11585,307.013146,1334,893
4,11585,319.807287,1220,893
5,11585,331.039145,892,893


In [18]:
# b is logsums
log = orca.get_table('logsums').to_frame()[['autoPeakTotal', 'transitPeakTotal' , 'autoOffPeakRetail']]
log.head()

,autoPeakTotal,transitPeakTotal,autoOffPeakRetail
taz,,,
1,13.0616,11.1182,10.5312
2,13.0557,10.8372,10.5185
3,13.0308,10.7516,10.4998
4,13.0801,11.1391,10.5458
5,13.0613,11.0483,10.5325


In [19]:
import pandas as pd
bb=pd.merge(bb, log, left_on='zone_id', right_index=True)
bb.head()

,node_id,residential_price,sqft_per_unit,zone_id,autoPeakTotal,transitPeakTotal,autoOffPeakRetail
building_id,,,,,,,
1,11584,392.395039,794,893,12.0346,6.8494,9.9296
2,11584,284.880588,1600,893,12.0346,6.8494,9.9296
3,11585,307.013146,1334,893,12.0346,6.8494,9.9296
4,11585,319.807287,1220,893,12.0346,6.8494,9.9296
5,11585,331.039145,892,893,12.0346,6.8494,9.9296


In [20]:
nn = orca.get_table('nodes').to_frame()[['ave_lot_size_per_unit','ave_income','ave_hhsize','jobs','pct_white','pct_black','pct_hisp','pct_asian']]
nn.head()

,ave_lot_size_per_unit,ave_income,ave_hhsize,jobs,pct_white,pct_black,pct_hisp,pct_asian
8,9.760910,10.962228,1.388186,7.051473,92.341011,39.740620,79.973503,53.366737
9,9.667663,10.947813,1.384454,7.129230,91.652740,29.368269,78.486153,43.259628
10,9.939272,10.901420,1.399366,7.164987,90.817871,0.000000,80.256088,0.000000
11,9.697163,10.905808,1.481605,7.112618,88.555183,14.350271,83.145981,14.350271
12,9.671385,10.884773,1.486836,7.108247,88.312546,15.288960,83.428207,15.288960


In [21]:
bb=pd.merge(bb, nn, left_on='node_id', right_index=True)
bb.head()

,node_id,residential_price,sqft_per_unit,zone_id,autoPeakTotal,transitPeakTotal,autoOffPeakRetail,ave_lot_size_per_unit,ave_income,ave_hhsize,jobs,pct_white,pct_black,pct_hisp,pct_asian
building_id,,,,,,,,,,,,,,,
1,11584,392.395039,794,893,12.0346,6.8494,9.9296,8.197712,10.517467,1.510924,6.793979,80.76889,90.016502,89.262566,75.130203
2,11584,284.880588,1600,893,12.0346,6.8494,9.9296,8.197712,10.517467,1.510924,6.793979,80.76889,90.016502,89.262566,75.130203
7,11584,379.145588,856,893,12.0346,6.8494,9.9296,8.197712,10.517467,1.510924,6.793979,80.76889,90.016502,89.262566,75.130203
28,11584,344.759122,1054,893,12.0346,6.8494,9.9296,8.197712,10.517467,1.510924,6.793979,80.76889,90.016502,89.262566,75.130203
29,11584,379.145588,856,893,12.0346,6.8494,9.9296,8.197712,10.517467,1.510924,6.793979,80.76889,90.016502,89.262566,75.130203


In [38]:
hh = orca.get_table('households').to_frame()
hh.head()

,taz,serialno,puma5,income,persons,hht,unittype,noc,bldgsz,tenure,...,income_quartile,positive_income,node_id,black,asian,hisp,white,hhs3p,hhs1,hh_monthly_rent
household_id,,,,,,,,,,,,,,,,,,,,,
227454,878,9279874,2404,17000,2,3,0,1,5,3,...,1,17000,NaN,1,0,0,0,False,False,0
2687058,1092,8097339,2103,0,1,0,2,0,0,0,...,1,1,NaN,0,0,0,0,False,True,0
2687063,1092,3530666,2103,0,1,0,2,0,0,0,...,1,1,NaN,0,0,0,0,False,True,0
2214782,930,5691854,2402,159700,3,5,0,0,2,1,...,4,159700,NaN,0,0,0,1,True,False,0
2687065,1092,8097339,2103,0,1,0,2,0,0,0,...,1,1,NaN,0,0,0,0,False,True,0


In [39]:
unit = orca.get_table('residential_units').to_frame()[['building_id']]
unit.head()

,building_id
unit_id,
0,1
1,2
2,3
3,4
4,5


In [40]:
hh=pd.merge(hh, unit, left_on='unit_id', right_index=True)
hh.head()

,taz,serialno,puma5,income,persons,hht,unittype,noc,bldgsz,tenure,...,positive_income,node_id,black,asian,hisp,white,hhs3p,hhs1,hh_monthly_rent,building_id_y
household_id,,,,,,,,,,,,,,,,,,,,,
982740,893,7685509,2404,52000,3,3,0,1,5,3,...,52000,11584,1,0,0,0,True,False,0.000000,1
1086165,893,3832158,2404,40650,6,1,0,4,3,2,...,40650,11584,0,0,1,0,True,False,1324.511619,2
1714074,893,7716621,2404,60000,6,1,0,4,2,1,...,60000,11585,1,0,0,0,True,False,2765.866270,3
180826,893,1733302,2404,28000,2,5,0,0,2,1,...,28000,11585,1,0,0,0,False,False,2420.016402,5
1048807,893,1090139,2404,50800,10,1,0,5,3,1,...,50800,11585,0,0,1,0,True,False,1519.696399,6


In [41]:
hh=pd.merge(hh, bb, left_on='unit_id', right_index=True)
hh.head()

,taz,serialno,puma5,income,persons,hht,unittype,noc,bldgsz,tenure,...,transitPeakTotal,autoOffPeakRetail,ave_lot_size_per_unit,ave_income,ave_hhsize,jobs,pct_white,pct_black,pct_hisp,pct_asian
household_id,,,,,,,,,,,,,,,,,,,,,
1086165,893,3832158,2404,40650,6,1,0,4,3,2,...,6.8494,9.9296,8.197712,10.517467,1.510924,6.793979,80.768890,90.016502,89.262566,75.130203
1714074,893,7716621,2404,60000,6,1,0,4,2,1,...,6.8494,9.9296,8.197712,10.517467,1.510924,6.793979,80.768890,90.016502,89.262566,75.130203
180826,893,1733302,2404,28000,2,5,0,0,2,1,...,6.8494,9.9296,8.211932,10.515741,1.510664,6.429195,80.199013,89.713486,89.017441,74.876244
1048807,893,1090139,2404,50800,10,1,0,5,3,1,...,6.8494,9.9296,8.211932,10.515741,1.510664,6.429195,80.199013,89.713486,89.017441,74.876244
381691,893,1505541,2404,17130,4,3,0,1,8,3,...,6.8494,9.9296,8.211932,10.515741,1.510664,6.429195,80.199013,89.713486,89.017441,74.876244


# OLS to get residual as a pandas series

In [42]:
hh['residential_price'].unique()

array([ 392.39503921,  284.88058795,  319.80728692, ...,  952.98110466,
        523.81349328,  406.71443663])

In [27]:
import numpy as np
import pandas
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [43]:
model = ols(" np.log1p(residential_price)~ np.log1p(sqft_per_unit)+ np.log1p(persons * ave_hhsize) + np.log1p(white *pct_white) + np.log1p(black * pct_black) + np.log1p(hisp * pct_hisp) + np.log1p(asian* pct_asian)+autoPeakTotal + jobs + transitPeakTotal + autoOffPeakRetail + np.log1p(ave_lot_size_per_unit)+ ave_income", data=hh).fit()

In [44]:
print model.summary()

                                 OLS Regression Results                                
Dep. Variable:     np.log1p(residential_price)   R-squared:                       0.105
Model:                                     OLS   Adj. R-squared:                  0.105
Method:                          Least Squares   F-statistic:                 1.709e+04
Date:                         Tue, 01 Mar 2016   Prob (F-statistic):               0.00
Time:                                 23:55:22   Log-Likelihood:            -3.2374e+06
No. Observations:                      1751796   AIC:                         6.475e+06
Df Residuals:                          1751783   BIC:                         6.475e+06
Df Model:                                   12                                         
Covariance Type:                     nonrobust                                         
                                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------

In [45]:
c = model.resid
#fillna(0)
print c.unique()

[ 0.61085598  0.71795495  0.62654219 ..., -5.11247627 -4.72505252
 -4.16035415]


In [46]:
hh['residual']=c
hh.head()

,taz,serialno,puma5,income,persons,hht,unittype,noc,bldgsz,tenure,...,autoOffPeakRetail,ave_lot_size_per_unit,ave_income,ave_hhsize,jobs,pct_white,pct_black,pct_hisp,pct_asian,residual
household_id,,,,,,,,,,,,,,,,,,,,,
1086165,893,3832158,2404,40650,6,1,0,4,3,2,...,9.9296,8.197712,10.517467,1.510924,6.793979,80.768890,90.016502,89.262566,75.130203,0.610856
1714074,893,7716621,2404,60000,6,1,0,4,2,1,...,9.9296,8.197712,10.517467,1.510924,6.793979,80.768890,90.016502,89.262566,75.130203,0.717955
180826,893,1733302,2404,28000,2,5,0,0,2,1,...,9.9296,8.211932,10.515741,1.510664,6.429195,80.199013,89.713486,89.017441,74.876244,0.626542
1048807,893,1090139,2404,50800,10,1,0,5,3,1,...,9.9296,8.211932,10.515741,1.510664,6.429195,80.199013,89.713486,89.017441,74.876244,0.469281
381691,893,1505541,2404,17130,4,3,0,1,8,3,...,9.9296,8.211932,10.515741,1.510664,6.429195,80.199013,89.713486,89.017441,74.876244,0.516730


## model correction

In [22]:
orca.add_column('households', 'residual', c)

In [23]:
orca.get_table('households').to_frame().describe()

,taz,serialno,puma5,income,persons,hht,unittype,noc,bldgsz,tenure,...,residual,positive_income,node_id,black,asian,hisp,white,hhs3p,hhs1,hh_monthly_rent
count,2732722.000000,2732722.000000,2732722.000000,2732722.000000,2732722.000000,2732722.000000,2732722.000000,2732722.000000,2732722.000000,2732722.000000,...,1.751796e+06,2732722.000000,2654829.000000,2732722.000000,2732722.000000,2732722.000000,2732722.000000,2732722,2732722,2732722.000000
mean,751.134174,4923397.817272,2168.537679,77670.162133,2.581065,2.640939,0.076833,0.468948,3.528608,1.893538,...,3.196574e-12,77673.227609,131652.891137,0.077111,0.166288,0.123539,0.655225,0.407958,0.290573,2819.494773
std,430.938788,2857497.525312,516.128215,81405.085003,1.605801,2.065958,0.365144,0.917211,2.516513,1.010827,...,1.535912e+00,81401.982998,91855.069436,0.266767,0.372339,0.329055,0.475295,0.491455,0.454027,3186.855611
min,1.000000,20.000000,1000.000000,-20000.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.222572e+01,1.000000,8.000000,0.000000,0.000000,0.000000,0.000000,False,False,0.000000
25%,372.000000,2463830.000000,2104.000000,26500.000000,1.000000,1.000000,0.000000,0.000000,2.000000,1.000000,...,2.123260e-01,26500.000000,58955.000000,0.000000,0.000000,0.000000,0.000000,0,0,0.000000
50%,762.000000,4901786.000000,2303.000000,58000.000000,2.000000,1.000000,0.000000,0.000000,2.000000,2.000000,...,4.067682e-01,58000.000000,126662.000000,0.000000,0.000000,0.000000,1.000000,0,0,2443.170110
75%,1144.000000,7361646.000000,2410.000000,100000.000000,4.000000,4.000000,0.000000,1.000000,5.000000,3.000000,...,5.747870e-01,100000.000000,172506.000000,0.000000,0.000000,0.000000,1.000000,1,1,4239.603453
max,1454.000000,9999899.000000,2714.000000,1968504.000000,25.000000,7.000000,2.000000,12.000000,10.000000,4.000000,...,8.137004e+00,1968504.000000,354681.000000,1.000000,1.000000,1.000000,1.000000,True,True,105840.633246


In [ ]:
nh=orca.get_table('households').to_frame()['residual'].unique()

In [76]:
orca.get_table('households').to_frame()['residual'].describe()

count    1.751796e+06
mean     3.196574e-12
std      1.535912e+00
min     -1.222572e+01
25%      2.123260e-01
50%      4.067682e-01
75%      5.747870e-01
max      8.137004e+00
Name: residual, dtype: float64

In [ ]:
df.fillna(0)

In [38]:
orca.get_injectable('unit_aggregations')[0].to_frame().columns

Index([u'parcel_id', u'residential_units', u'residential_sqft',
       u'non_residential_sqft', u'building_sqft', u'stories', u'year_built',
       u'redfin_sale_year', u'building_type_id', u'residential_price',
       u'non_residential_price', u'zone_id', u'general_type', u'sqft_per_job',
       u'residential_rent', u'residual', u'lot_size_per_unit',
       u'vacant_job_spaces', u'vacant_residential_units', u'node_id',
       u'base_price_per_sqft', u'job_spaces', u'sqft_per_unit'],
      dtype='object')

## best model for price

In [47]:
orca.add_table('new_h', hh)

In [48]:
orca.get_table('new_h').to_frame()['residual'].unique()

array([ 0.61085598,  0.71795495,  0.62654219, ..., -5.11247627,
       -4.72505252, -4.16035415])

In [49]:
@orca.step("hlcm_owner_estimate_NEW11")
def hlcm_owner_estimate_NEW(new_h, residential_units, unit_aggregations):
    return utils.lcm_estimate("hlcm_owner_NEW11.yaml", new_h, "unit_id",
                              residential_units, unit_aggregations)

orca.run(["hlcm_owner_estimate_NEW11"])

Running step 'hlcm_owner_estimate_NEW11'
LCM RESULTS FOR SEGMENT 1

Null Log-liklihood: -69077.553
Log-liklihood at convergence: -65796.747
Log-liklihood Ratio: 0.047

+---------------------------------+-------------+------------+---------+
| Component                       | Coefficient | Std. Error | T-Score |
+---------------------------------+-------------+------------+---------+
| np.log1p(residential_price)     |    2.801    |   0.094    |  29.702 |
| np.log1p(sqft_per_unit)         |    1.185    |   0.047    |  25.456 |
| np.log1p(ave_lot_size_per_unit) |    -2.310   |   0.135    | -17.174 |
| residual                        |    -0.000   |   0.005    |  -0.000 |
| ave_income                      |    -1.444   |   0.054    | -26.756 |
| np.log1p(persons * ave_hhsize)  |    2.371    |   0.022    | 106.613 |
| np.log1p(white * pct_white)     |    3.000    |   0.009    | 331.655 |
| np.log1p(black * pct_black)     |    3.000    |   0.011    | 276.138 |
| np.log1p(hisp * pct_hisp)  

In [50]:
orca.run(["hlcm_owner_estimate"])

Running step 'hlcm_owner_estimate'
LCM RESULTS FOR SEGMENT 1

Null Log-liklihood: -68156.519
Log-liklihood at convergence: -65678.718
Log-liklihood Ratio: 0.036

+---------------------------------+-------------+------------+---------+
| Component                       | Coefficient | Std. Error | T-Score |
+---------------------------------+-------------+------------+---------+
| np.log1p(residential_price)     |    2.438    |   0.091    |  26.770 |
| np.log1p(sqft_per_unit)         |    1.089    |   0.045    |  24.118 |
| np.log1p(ave_lot_size_per_unit) |    -0.226   |   0.147    |  -1.540 |
| ave_income                      |    -1.623   |   0.049    | -33.045 |
| np.log1p(persons * ave_hhsize)  |    3.000    |   0.022    | 134.421 |
| np.log1p(white * pct_white)     |    3.000    |   0.009    | 343.969 |
| np.log1p(black * pct_black)     |    3.000    |   0.011    | 268.135 |
| np.log1p(hisp * pct_hisp)       |    3.000    |   0.007    | 418.679 |
| np.log1p(asian * pct_asian)     |